# Spark Data Wrangling

In [1]:
import pandas as pd
import os

# from sqlalchemy import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 

from env import user, password, host

# shhhhh..
import warnings
warnings.filterwarnings('ignore')

In [2]:
#create enviroment
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/21 12:49:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def get_db_url(db):
    return (f'mysql+pymysql://{user}:{password}@{host}/{db}')

In [4]:
def get_311_data():
    filename = '311_data.csv'

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        df = pd.read_sql("""SELECT * FROM source""", get_db_url('telco_churn'))
        df.to_csv(filename)
        return df

## Acquire

In [36]:
cases_query = '''SELECT * FROM cases'''
url = get_db_url("311_data")

In [37]:
cases_df = pd.read_sql(cases_query, url)

In [38]:
cases_df = spark.createDataFrame(cases_df)
cases_df.show(4)

22/10/21 14:30:01 WARN TaskSetManager: Stage 4 contains a task of very large size (12594 KiB). The maximum recommended task size is 1000 KiB.


22/10/21 14:30:05 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 4 (TID 13): Attempting to kill Python Worker
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 

### load mpg data set from pydataset

In [9]:
from pydataset import data

### write datafame to file

- `json`: for writing to a local json file(s)
- `csv`: for writing to a local csv file(s)
- `parquet`: Parquet is a very popular columnar storage format for Hadoop.
- `jdbc`: for writing to a SQL database table

#### write file to json

In [10]:
#df.write.type


#### write dataframe to csv

In [11]:
#df.write.format()


### read files
- spark.read.[type]

#### read json

#### read csv

### load source from 311_data in sql

In [12]:
#sql query


In [13]:
#make pandas df


In [14]:
#create spark df


### data schema

In [15]:
# from pyspark.sql.types import StructType, StructField, StringType

In [16]:
#use structype to build a list of the structfield with the columns assocaited schema


### load cases from 311_data from sql

In [17]:
#sql query


In [18]:
#pandas df


In [19]:
#spark df


## Prepare

- rename columns
- correct datatypes
- data transformation
- make new features
- join tables

### rename columns

#### change SLA_due_date to case_due_date

### correct datatypes

#### change close_closed and case_late columns into boolean values

In [20]:
#use condition to make true and false


#### change council_district datatype to string

In [21]:
#use .cast()


#### change dates to datetype

format date strings: https://docs.oracle.com/javase/10/docs/api/java/time/format/DateTimeFormatter.html

In [22]:
#use to_timestamp


### data transformation

#### normalize address
- `lower`: lowercase everything
- `trim`: remove whitespace on the edges 

#### change num_days_late to num_weeks_late

#### change council_district to int and pad with 00s

In [23]:
# '%03d' means at least 3 digits, pad with 0s


### new features

#### create zip code column

#### create case_lifetime column

- case_age: how long since the case first opened
- days_to_close: the number of days between days opened and days closed
- case_lifetime: if the case is open, how long since the case opened, if the case is closed, the number of days to close


In [24]:
#use datediff() to find the difference between two dates


In [25]:
#create days_to_close


In [26]:
#look at new columns for cases that were closed


In [27]:
#look at new columns for cases that were NOT closed


In [28]:
#create case_lifetime column


In [29]:
#drop unnecessary columns


### join the dept table from sql to our current df

In [30]:
#get dept table from sql


In [31]:
#join tables


### train, validate, test split

- `.randomSplit` to split df